In this project, I am going to use spacy for entity recognition on 200 Resume 
and experiment around various NLP tools for text analysis. 

I have also added skills match feature so that hiring managers can follow a metric that will
help them to decide whether they should move to the interview stage or not. 

I will be using two datasets; the first contains resume texts 
and the second contains skills that we will use to create an entity ruler.

## Inside the CSV
* ID: Unique identifier and file name for the respective pdf.
* Resume_str : Contains the resume text only in string format.
* Resume_html : Contains the resume data in html format as present while web scrapping.
* Category : Category of the job the resume was used to apply.

## Present categories
HR, Designer, Information-Technology, Teacher, Advocate, Business-Development, Healthcare, Fitness, Agriculture, BPO, Sales, Consultant, Digital-Media, Automobile, Chef, Finance, Apparel, Engineering, Accountant, Construction, Public-Relations, Banking, Arts, Aviation

### Jobzilla skill patterns

The jobzilla skill dataset is jsonl file containing different skills that can be used to create spaCy entity_ruler. 

The data set contains label and pattern-> diferent words used to descibe skills in various resume.

In [2]:
!pip install spacy

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\Eya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [10]:
!pip install pyLDAvis

  Using cached pyLDAvis-3.3.1-py2.py3-none-any.whl
  Using cached funcy-1.16-py2.py3-none-any.whl (32 kB)
  Using cached numexpr-2.8.0-cp39-cp39-win_amd64.whl (88 kB)
  Using cached sklearn-0.0-py2.py3-none-any.whl


In [12]:
!pip install wordcloud

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wordcloud: filename=wordcloud-1.8.1-cp39-cp39-win_amd64.whl size=161111 sha256=276327029c8eb6dcb40f4d61cf433f546765813cade9ddc3ec2a2c57beb27324
  Stored in directory: c:\users\eya\appdata\local\packages\pythonsoftwarefoundation.python.3.9_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\f9\7a\dd\06ef8b5dfe5483f6204133c08eeb16c287cc2c05e290ae2fc0
Successfully built wordcloud


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [14]:
!pip install jsonlines

In [16]:
!pip install nltk

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [17]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Eya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


## Loading
In this section, I am going to load the spaCy model, Resume Dataset, and Jobzilla skills dataset directly into the entity ruler.

### Resume Dataset
Using Pandas read_csv to read dataset containing text data about Resume.

* I am going to randomized Job categories so that 200 samples contain various job categories instead of one.
* I am going to limit our number of samples to 200 as processing 2400+ takes time.

In [19]:
df = pd.read_csv("../Getting The Data/Resume.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[0:200,]
data.head()

,ID,Resume_str,Resume_html,Category
1900,24703009,STAFF ACCOUNTANT Virginia F...,"<div class=""fontsize fontface vmargins hmargin...",ACCOUNTANT
2223,17213671,PARK OPERATIONS HOSTESS Summa...,"<div class=""fontsize fontface vmargins hmargin...",BANKING
2426,25718772,TSO/FLOATER Career Overview ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
1455,29990140,PROJECT MANAGER AND PHOTOGRAPHER ...,"<div class=""fontsize fontface vmargins hmargin...",CHEF
848,56286525,OPERATIONS MANAGER Executive Su...,"<div class=""fontsize fontface vmargins hmargin...",FITNESS


### Loading spaCy model
I can download spaCy model then load spacy model into nlp.

In [24]:
nlp = spacy.load("en_core_web_sm")
skill_pattern_path = "jz_skill_patterns.jsonl"

### Entity Ruler
To create an entity ruler we need to add a pipeline and then load the .jsonl file containing skills into ruler.

As you can see we have successfully added a new pipeline entity_ruler. 

Entity ruler helps us add additional rules to highlight various categories within the text, such as skills and job description in our case.

In [27]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

ValueError: [E007] 'entity_ruler' already exists in pipeline. Existing names: ['tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'ner', 'entity_ruler']

### Skills

I will create two python functions to extract all the skills within a resume and create an array containing all the skills. 

Later I am going to apply this function to the dataset and create a new feature called skill. 

This will help us visualize trends and patterns within the dataset.

get_skills is going to extract skills from a single text.
unique_skills will remove duplicates.

In [28]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))